## Training notebook for SMART 3.0

This notebook is written to replicate the training of SMART 3.0 model with Edited-HSQC data.  
It is simplified just for the replicating the model.

System Environment

tensorflow == 2.3.0 (with CUDA 10.1)  
keras == 2.4.0  
numpy == 1.18.5  
json == 2.0.9  
requests == 2.22.0

In [5]:
#Loding Package
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models

import numpy as np
import pandas as pd
K = tf.keras.backend

import random
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from requests import get

## Loading data
Two kind of data are essential for training the model.  
Dataset contains HSQC data, finger prints, molecular weights, class, and glycoside information.  
Index file contains the classificaion ontology from NPClassifier.  
(https://chemrxiv.org/articles/preprint/NPClassifier_A_Deep_Neural_Network-Based_Structural_Classification_Tool_for_Natural_Products/12885494/1)

dataset size == 210 MB  
index_data == 56 KB

In [4]:
url_dataset = 'https://www.dropbox.com/s/fa5if10971cm6sz/Training_set_1.npy?dl=1'
url_index = 'https://www.dropbox.com/s/faqydz94mo2xy4d/index_v1.json?dl=1'
r = get(url_dataset, allow_redirects=True)
open('dataset.npy', 'wb').write(r.content)
r = get(url_index, allow_redirects=True)
open('index_v1.json', 'wb').write(r.content)

dataset = np.load('dataset.npy', allow_pickle=True) # dataframe, FP, Class_No, and Glycoside
with open('index_v1.json','rb') as r:
    index = json.load(r)

In [4]:
#Data split for training and validation (stratified data split based on the compound classes)
train_set, validation_set = train_test_split(dataset, test_size = 0.2, random_state=42,stratify=dataset[:,3] )

In [6]:
def data_preparation(data):
    images = np.zeros((len(data),128,128,2),int)
    fps = np.zeros((len(data),6144),int)
    mws = np.zeros((len(data),1),int)
    clss = np.zeros((len(data),1),int)
    glys = np.zeros((len(data),1),int)
    for i in range(len(data)):
        image = np.array(data[i][0])
        for l in range(len(image)):
            a, b, t = image[l][0], image[l][1], image[l][2]
            try:
                if t>=0 and 0 <= a < 128 and 0 <= b < 128:
                    images[i, a, scale-b-1,0] = 1
                elif t<0 and 0 <= a < 128 and 0 <= b < 128:    
                    images[i, a, scale-b-1,1] = 1
            except:
                continue
        
        fp = data[i][1]
        for bit in fp:
            fps[i][bit] = 1
        
        mws[i] = data[i][2]
        
        clss[i] = data[i][3]
        glys[i] = data[i][4]
        
    clss =  tf.keras.utils.to_categorical(clss,num_classes=len(index['Superclass']))
    glys = tf.keras.utils.to_categorical(glys,num_classes=2)
    
    # This outline improved the performance by giving spatial information of peaks on the HSQC spectra
    images[:,:,0,:] = 1
    images[:,:,127,:] = 1
    images[:,0,:,:] = 1
    images[:,127,:,:] = 1
    
    output = [fps, mws, clss, glys]
    return images, output

In [7]:
def masked_CE():
    # Not all compound are fully classified by NPClassifier.
    # So those unclassified compounds were labelled as '-1' and this unclassified compounds were masked and not utilized during the backprogation.
    def loss(y_true, y_pred):
        y_true_masked = tf.boolean_mask(y_true, tf.reduce_any(tf.not_equal(y_true, -1), 1))
        y_pred_masked = tf.boolean_mask(y_pred, tf.reduce_any(tf.not_equal(y_true, -1), 1))
        loss = K.mean(K.categorical_crossentropy(y_true_masked, y_pred_masked))
        return loss
    
    return loss

def build_model():
    def residual_block(X,n):
        X_shortcut = X
        X = layers.BatchNormalization()(X)
        X = layers.Activation('relu')(X)
        X = layers.Conv2D(n, (3, 3), padding="same")(X)
        X = layers.BatchNormalization()(X)
        X = layers.Activation('relu')(X)
        X = layers.Conv2D(n, (3, 3), padding="same")(X)
        X = layers.BatchNormalization()(X)
        X = layers.Activation('relu')(X)
        X = layers.Conv2D(n, (3, 3), padding="same")(X)
        X = layers.BatchNormalization()(X)
        X = layers.Activation('relu')(X)
        X = layers.Concatenate()([X,X_shortcut])
        return X

    Input_image = layers.Input(shape=(128,128,2))
    X = layers.Conv2D(64, (3, 3), padding="same")(Input_image)
    X = layers.BatchNormalization()(X)
    X = layers.Activation('relu')(X)
    X = layers.Conv2D(64, (3, 3), padding="same")(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation('relu')(X)
    X = layers.MaxPooling2D((2, 2))(X)
    X = residual_block(X,128)
    X = layers.MaxPooling2D((2, 2))(X)
    X = residual_block(X,256)
    X = layers.MaxPooling2D((2, 2))(X)
    X = residual_block(X,512)
    X = layers.MaxPooling2D((2, 2))(X)
    X = residual_block(X,1024)
    X = layers.MaxPooling2D((2, 2))(X)
    X = residual_block(X,2048)
    X = layers.GlobalMaxPooling2D()(X)
    X = layers.Dropout(0.2)(X)


    FP_output = layers.Dense(6144, activation = 'sigmoid', name = 'FP')(X)
    MW_output = layers.Dense(1, name = 'MW')(X)
    output1 = layers.Dense(len(index['Superclass']), activation = 'softmax', name = 'class')(X)
    output2 = layers.Dense(2, activation = 'softmax',name='glycoside')(X)

    model = keras.Model(inputs = Input_image, outputs = [FP_output, MW_output, output1, output2])

    model.compile(optimizer=keras.optimizers.Adam(lr=0.00001),loss=['binary_crossentropy','MAPE',masked_CE(),masked_CE()],loss_weights = [10000,1,1,1],metrics={'FP':'cosine_proximity','MW':'MAPE','class':'accuracy','glycoside':'accuracy'})
    return model

#Buiding model.
model = build_model()

In [15]:
# Generating the tensor
train_x , train_y = data_preparation(train_set)
val_x, val_y = data_preparation(validation_set)

In [ ]:
# trainig the model.
model.fit(x=train_x,y=train_y,
        validation_data=(val_x,val_y),batch_size=16, epochs=30,
        use_multiprocessing=False, verbose=1)